In [52]:
import sys
import simulator as sim
import numpy as np
import datetime as dt
import pandas as pd

SEC = 1000000
MIN = 60 * SEC
DOLLAR = 1000000

DATA_PATH = "data/"
###############################################################################
# Read from book data file and convert it to feedable SVM data
###############################################################################
    
    
def treat_data(ticker, date, threshold = 0.01):
    raw_filename = "{0}_{1}_bookdata.txt".format(ticker, date) 
    raw_data = np.loadtxt(raw_filename)
    num_data = raw_data.shape[0]
    num_lvls = (raw_data.shape[1] - 2) / 2
    buildup = [1, 2, 3, 5, 10, 15]
    
    # sig_data is the output signal for the SVM, as a (n, 1) ndarray
    sig_data = np.empty(shape = [num_data - 15, 1])
    # obp_data is the SVM input as a (n, 6 * num_lvls) ndarray where
    # OBP(t, l) = obp_data[ , t*num_lvls + + l]
    obp_data = np.empty(shape = [num_data - 15, 6 * num_lvls])
    
    for i in range(15, num_data):
        price_change = (raw_data[i, 1] - raw_data[i - 1, 1]) / raw_data[i - 1, 1]
        if price_change < -threshold:
            sig_data[i-15, 0] = -1
        elif price_change > threshold:
            sig_data[i-15, 0] = 1
        else:
            sig_data[i-15, 0] = 0
            
        for t in range(len(buildup)):
            for j in range(num_lvls):
                obp_data[i-15, t*num_lvls + j] = np.sum(raw_data[(i-buildup[t]):(i+1), 7:(j+8)]) / np.sum(raw_data[(i-buildup[t]):(i+1), 2:(j+3)])
                            
    obp_filename = "{0}_{1}_obp.txt".format(ticker, date)
    sig_filename = "{0}_{1}_sig.txt".format(ticker, date)
    np.savetxt(obp_filename, obp_data, newline = "\n")
    np.savetxt(sig_filename, sig_data, newline = "\n")
    print "Saved to {0}".format(obp_filename)
    
    
###############################################################################
# Default algorithm with best bid best ask
###############################################################################


class TestAlgo:
    
    def __init__(self, session, date, ticker, start_time, end_time):
        # Save session information
        self.session = session
        self.date = date
        self.tickers = ticker
        self.start_time = start_time
        self.end_time = end_time
        self.order_size = 1
        # PnL and book value functions need to be tested when order_size is not 1
        self.mu = 0
        self.ORDER_WAIT = 30
        print "date is {0}".format(date)
        # outstanding_orders is a dictionary, tickers are the keys and values are 
        # {"A": [ask_order_id, time_accepted], "B": [bid_order_id, time_accepted]}
        # if there are outstanding orders
        self.outstanding_orders = {}
        
        # signal is a dictionary : {symbol : symbol_signal}
        self.signal = {}
        
        # inventory is another dictionary: {symbol : inventory_size for symbol}
        # cash is another dictionary: {symbol : cash_balance for symbol}
        # bookvalue is another dictionary: {symbol : book value for symbol}
        self.inventory = {}
        self.cash = {}
        self.bookvalue = {}
        # self.inv_hist is a dict of numpy arrays, the arrays are as follows:
        # field for each column: time, symbol_cash, symbol_inventory, symbol_book_value, symbol_current_mid_price
        self.inv_hist = {}
        
        # Suscribe to tickers and set first timer
        # no trading on the 1st minute: use to compute signal
        num = (end_time - start_time) / MIN
        self.obs_counter = 0
        for i in range(num):
            self.session.add_timer(self.start_time + (i + 1) * MIN, self.timer_OBS_callback)
        
        for sym_it in self.tickers:
            self.session.subscribe_ticker_all_feeds(sym_it)
            self.outstanding_orders[sym_it] = {}
            self.signal[sym_it] = self.get_signal(sym_it)
            self.inventory[sym_it] = 0
            self.cash[sym_it] = 0
            self.bookvalue[sym_it] = 0
            self.session.subscribe_event(sym_it, sim.ORDER_ACCEPTED, self.order_ack_callback)
            self.session.subscribe_event(sym_it, sim.ORDER_EXECUTED, self.order_exe_callback)
            self.session.subscribe_event(sym_it, sim.ORDER_CANCELED, self.cancelled_callback)
            self.session.subscribe_event(sym_it, sim.ORDER_REJECTED, self.rejected_callback)
            self.inv_hist[sym_it] = np.zeros((num, 5))
        
        
        
    def timer_OBS_callback(self, time):
        #print "OBS callback: {0}, {1}".format(sim.micro_to_time(time), sim.micro_to_time(self.end_time))
        if (time >= self.end_time):
            self.end()
            return
        for sym_it in self.signal:
            curr_sig = self.get_signal(sym_it)
            if (curr_sig != self.signal[sym_it]):
                cancel_orders(sym_it)
                stop_loss(sym_it)
                self.signal[sym_it] = curr_sig
            self.post_orders(sym_it)
        for sym_it in self.tickers:
            bid, ask = self.session.get_inside_market(sym_it)
            b_price = bid['price']
            a_price = ask['price']
            self.inv_hist[sym_it][self.obs_counter] = np.array([time, self.cash[sym_it], self.inventory[sym_it], self.bookvalue[sym_it], (a_price + b_price) / 2])
        
        self.obs_counter += 1    
            
            
    def post_orders(self, ticker):
        now = self.session.current_time()
        orders = self.session.get_orders_by_ticker(ticker)
        if not orders:
            bid, ask = self.session.get_inside_market(ticker)
            b_price = bid['price'] # = bestBidPrice()
            a_price = ask['price']
            self.session.add_order(ticker, sim.BUY, self.order_size, b_price + self.signal[ticker] * self.mu, exchange=sim.EXCH_INET)
            self.session.add_order(ticker, sim.SELL, self.order_size, a_price + self.signal[ticker] * self.mu, exchange=sim.EXCH_INET) 
            #print "Posted orders"
               
    
    def cancel_orders(self, ticker):
        now = self.session.current_time()
        #print "cancel order call back: it's now {0}".format(sim.micro_to_time(now))
        ord_info = self.outstanding_orders[ticker]
        if len(ord_info) == 0:
            #print "no order to cancel"
            return
        else:
            for side in ord_info:
                sim.cancel_order(ord_info[side][0])
                #print "cancelled order {0}".format(ord_info[side][0])
        
        
    def stop_loss(self, ticker):
        print "stop loss for ticker {0}".format(ticker)
        inv = self.inventory[ticker]
        if (inv == 0):
            return
        elif (inv > 0):
            # not the right way to send market order
            self.session.add_order(ticker, sim.SELL, inv, 1 * DOLLAR, exchange=sim.EXCH_INET)
            print "selling inventory"
        else:
            # not the right way to send market order
            self.session.add_order(ticker, sim.BUY, -inv, 999 * DOLLAR, exchange=sim.EXCH_INET)
            print "buying inventory"
        
        
    def timer_order_manage_callback(self, time):
        now = self.session.current_time()
        #print "order manage back: it's now {0}".format(sim.micro_to_time(now))
        for sym_it in self.outstanding_orders:
            ord_info = self.outstanding_orders[sym_it]
            if len(ord_info) == 0:
                continue
            else:
                for side in ord_info:
                    cancel_time = ord_info[side][1]
                    if (now < cancel_time):
                        #this may add duplicate timer event for different symbols
                        self.session.add_timer(cancel_time, self.timer_order_manage_callback)
                    else:
                        sim.cancel_order(ord_info[side][0])
                        #print "Cancel order {0}".format(ord_info[side][0])
         
        
    def rejected_callback(self, ticker, event_params):
        print "rejected!!!!"
        print event_params['rejected_orders']
    
    
    def order_ack_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Ack Call back: it's now {0}".format(sim.micro_to_time(time))
        for order in event_params['accepted_orders']:
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_accepted']
            self.outstanding_orders[sym][side] = [ord_id, t]
        #print "Order accepted. order id is {0}, symbol is {1}, side is {2}, time is {3}".format(ord_id, sym, side, sim.micro_to_time(time))
        if time < self.end_time:
            self.session.add_timer(t + self.ORDER_WAIT * SEC, self.timer_order_manage_callback)    
        
        
    def order_exe_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Execution call_back: it's now {0}".format(sim.micro_to_time(time))
        for exe in event_params['executed_orders']:
            order = exe['order']
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_placed']
            self.outstanding_orders[sym].pop(side)
            if (side == "B"):
                if self.inventory[sym] < 0 :
                    if -self.inventory[sym] >= exe['quantity_executed']:
                        proportion = exe['quantity_executed'] / float(-self.inventory[sym])
                        self.bookvalue[sym] -= self.bookvalue[sym] * proportion
                    else:
                        self.bookvalue[sym] = (self.inventory[sym] + exe['quantity_executed']) * exe['price_executed']
                else:
                    self.bookvalue[sym] += exe['quantity_executed'] * exe['price_executed']
                self.inventory[sym] += exe['quantity_executed'] 
                self.cash[sym] -= exe['quantity_executed'] * exe['price_executed']
                if time < self.end_time:
                    if "S" in self.outstanding_orders[sym]:
                        self.outstanding_orders[sym]["S"][1] = time + self.ORDER_WAIT * SEC
                        #print "extending timer for sell order"
            else:
                if self.inventory[sym] > 0 :
                    if self.inventory[sym] >= exe['quantity_executed']:
                        proportion = exe['quantity_executed'] / float(self.inventory[sym])
                        self.bookvalue[sym] -= self.bookvalue[sym] * proportion
                    else:
                        self.bookvalue[sym] = (self.inventory[sym] - exe['quantity_executed']) * exe['price_executed']
                else:
                    self.bookvalue[sym] -= exe['quantity_executed'] * exe['price_executed']
                self.inventory[sym] -= exe['quantity_executed'] 
                self.cash[sym] += exe['quantity_executed'] * exe['price_executed']
                if time < self.end_time:
                    if "B" in self.outstanding_orders[sym]:
                        self.outstanding_orders[sym]["B"][1] = time + self.ORDER_WAIT * SEC
                        #print "extending timer for buy order"
            #print "Order Executed. order id is {0}, symbol is {1}, side is {2}, time is {3}, dollar amount is {4}".format(ord_id, sym, side, sim.micro_to_time(time), exe['quantity_executed'] * exe['price_executed'])
    
    def cancelled_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Cancelled call back: it's now {0}".format(sim.micro_to_time(time))
        for order in event_params['canceled_orders']:
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_placed']
            self.outstanding_orders[sym].pop(side)
            #print "Order Cancelled. order id is {0}, symbol is {1}, side is {2}, time is {3}".format(ord_id, sym, side, sim.micro_to_time(time))
        
        
    #returns -1, 0, or 1 corresponding to markets going up, even, or down
    def get_signal(self, ticker):
        return 0
    
    
    def end_algo(self, time):
        for sym_it in self.cash:
            print "PnL for {0} is {1}".format(sym_it, self.cash[sym_it] / float(DOLLAR))
        
        for sym_it in self.tickers:
            bid, ask = self.session.get_inside_market(sym_it)
            b_price = bid['price']
            a_price = ask['price']
            self.inv_hist[sym_it][self.obs_counter] = np.array([time, self.cash[sym_it], self.inventory[sym_it], self.bookvalue[sym_it], (a_price + b_price) / 2])
        
        obp_filename = "{0}{1}_output".format(DATA_PATH, self.date)
        np.savez_compressed(obp_filename, **self.inv_hist)
    
    def end(self):
        time = self.session.current_time()
        if time < sim.string_to_micro("18:00"):
            print "DONDONDON: it's now {0}".format(sim.micro_to_time(time))
            print "Done"
            for sym_it in self.outstanding_orders:
                self.cancel_orders(sym_it)
            print "cash: {0}".format(self.cash)
            for sym_it in self.tickers:
                print "inventory: {0}".format(self.inventory[sym_it])    
                self.stop_loss(sym_it)

            self.session.add_timer(time + MIN, self.end_algo)
            
        return 

    

###############################################################################
# Main
###############################################################################


#date = "20150121"
start = dt.date( 2015, 1, 21 )
end = dt.date( 2015, 2, 12 )
days = pd.bdate_range(start - dt.timedelta(days=10), end)
to_remove = ["20150119", "20150216", "20150403", "20150525", "20150703", "20150824", "20151126", "20151225",
             "20160101","20160118","20160215","20160325","20160530","20160704","20160905","20161124","20161226",
             "20170102","20170116","20170220","20170414","20170518"]
#days = days[!days['date'].isin(to_remove)]
symbols = ["IVV"]
start_time = sim.string_to_micro("10:00")
end_time = sim.string_to_micro("10:10")

#simul_storedata = sim.Simulator(StoreDayData)
#simul_storedata.run("20150120", symbols, num_levels = 5, start_time = start_time, end_time = end_time)
for day in days:
    training_date = '{:%Y%m%d}'.format(day.date())
    #print training_date
    if training_date in to_remove:
        continue
    #simul_storedata = sim.Simulator(StoreDayData)
    #simul_storedata.run(training_date, symbols, num_levels = 5, start_time = start_time, end_time = end_time)
    
simul_trading = sim.Simulator(TestAlgo)
simul_trading.run('20150112', symbols, use_om=True, start_time=start_time, end_time=end_time)

date is 20150112
DONDONDON: it's now 10:10:00.000129
Done
cash: {'IVV': -203780000}
inventory: 1
stop loss for ticker IVV
selling inventory
PnL for IVV is 0.04


In [53]:
t = np.load('data/20150112_output.npz')
print t.files
print t['IVV']

['IVV']
[[  3.60600000e+10   0.00000000e+00   0.00000000e+00   0.00000000e+00
    2.03660000e+08]
 [  3.61200000e+10  -2.03640000e+08   1.00000000e+00   2.03640000e+08
    2.03555000e+08]
 [  3.61800000e+10  -2.03610000e+08   1.00000000e+00   2.03540000e+08
    2.03435000e+08]
 [  3.62400000e+10  -1.60000000e+05   0.00000000e+00   0.00000000e+00
    2.03675000e+08]
 [  3.63000000e+10  -2.03820000e+08   1.00000000e+00   2.03660000e+08
    2.03705000e+08]
 [  3.63600000e+10  -2.03790000e+08   1.00000000e+00   2.03675000e+08
    2.03755000e+08]
 [  3.64200000e+10  -2.03760000e+08   1.00000000e+00   2.03707500e+08
    2.03755000e+08]
 [  3.64800000e+10   1.00000000e+04   0.00000000e+00   0.00000000e+00
    2.03840000e+08]
 [  3.65400000e+10  -2.03810000e+08   1.00000000e+00   2.03820000e+08
    2.03815000e+08]
 [  3.66600001e+10   4.00000000e+04   0.00000000e+00   0.00000000e+00
    2.03880000e+08]]
